# Ranking: Banking Sector
1. Profit: ROE, ROA, NIM
2. Health: Loan-to-provision; Loan-to-deposit; Non-performing Loan; NPL Coverage

## 1. Import 

### 1.1 Library

In [32]:
import pandas as pd
import numpy as np
import datetime as dt
import pyodbc
import pymssql
import sys

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Customize the display of the table
pd.set_option('chained_assignment', None)

In [33]:
# Get today
now = dt.datetime.today().strftime("%Y%m%d")
now

'20230811'

### 1.2 Get the result from `ptsp_stock_fundamental_score`

In [34]:
# Get raw final result
conn = pyodbc.connect(
    r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=V:\iBroker\stock_database.accdb;'
)
df_raw = pd.read_sql('select * from ptsp_stock_fundamental_score', conn)

conn.close()

In [35]:
df_raw

,Symbol,Year,Quarter,Date,score_ROE_sector,score_ROA_sector,score_ROS_sector,score_ROE_group,score_ROA_group,score_ROS_group,...,rank_growth,score_PE_5Y,score_PB_5Y,score_PE_sector,score_PB_sector,score_valuation,rank_valuation,score_final,rank_final,Update
0,AAA,2021,4,2022-04-20,0.0,0.0,0.0,0.0,1.0,0.0,...,D,0.0,2.0,2.0,0.0,2.0,C,1.75,C,20220422
1,AAM,2021,4,2022-04-20,0.0,0.0,0.0,0.0,0.0,0.0,...,C,0.0,1.0,0.0,0.0,0.5,D,0.875,D,20220422
2,AAS,2021,4,2022-04-20,1.0,1.0,None,1.0,1.0,None,...,B,0.0,0.0,0.0,1.0,0.5,D,2.375,B,20220422
3,AAT,2021,4,2022-04-20,0.5,0.5,0.5,0.5,0.5,0.5,...,B,0.0,0.0,2.0,2.0,2.0,C,2.5,B,20220422
4,AAV,2021,4,2022-04-20,0.0,0.0,0.0,0.0,0.0,0.0,...,C,0.0,0.0,0.0,2.0,1.0,D,1.75,C,20220422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18106,VTV,2023,1,2023-06-26,0.0,0.0,0.0,0.0,0.0,0.0,...,D,2.0,1.0,0.0,0.0,1.5,C,1.625,C,20230627
18107,VTX,2023,1,2023-06-26,0.0,0.0,0.0,0.0,0.0,1.0,...,C,0.0,1.0,0.0,0.0,0.5,D,1.375,C,20230627
18108,VTZ,2023,1,2023-06-26,0.0,0.0,0.0,1.0,1.0,0.0,...,A,0.0,0.0,0.0,0.0,0.0,D,1.75,C,20230627
18109,VUA,2023,1,2023-06-26,0.0,0.0,None,0.0,0.0,None,...,D,None,0.0,None,0.0,0.0,D,1.25,C,20230627


### 1.3 Process data following new model

In [36]:
df = pd.read_csv(r"F:\Tùng\Tung\Python\BSC_DataRankingStocks\cache\list_banks.csv")
list_banks = df['Symbol']
df_bank = df_raw.loc[df_raw['Symbol'].isin(list_banks)]

df_bank = df_bank[['Symbol', 'Year', 'Quarter', 'score_ROE_sector',
       'score_ROA_sector', 'score_NIM_sector', 'score_profit', 'rank_profit', 
       'z_LoanProvisionRatio', 'z_Deposit2Loan', 'z_NPL_ratio_inv',
       'z_NPL_coverage', 'score_health', 'rank_health',
       'score_EPS_above_average', 'score_EPS_growth', 'score_EPS_above_sector',
       'score_EPS_above_group', 'score_growth', 'rank_growth', 'score_PE_5Y',
       'score_PB_5Y', 'score_PE_sector', 'score_PB_sector', 'score_valuation',
       'rank_valuation', 'score_final', 'rank_final', 'Update']]

df_bank[[
    'score_ROE_sector', 'score_ROA_sector', 'score_NIM_sector', 'score_profit',
    'z_LoanProvisionRatio', 'z_Deposit2Loan', 'z_NPL_ratio_inv',
    'z_NPL_coverage', 'score_health', 'score_EPS_above_average',
    'score_EPS_growth', 'score_EPS_above_sector', 'score_EPS_above_group',
    'score_growth', 'score_PE_5Y', 'score_PB_5Y', 'score_PE_sector',
    'score_PB_sector', 'score_valuation', 'score_final'
]] = df_bank[[
    'score_ROE_sector', 'score_ROA_sector', 'score_NIM_sector', 'score_profit',
    'z_LoanProvisionRatio', 'z_Deposit2Loan', 'z_NPL_ratio_inv',
    'z_NPL_coverage', 'score_health', 'score_EPS_above_average',
    'score_EPS_growth', 'score_EPS_above_sector', 'score_EPS_above_group',
    'score_growth', 'score_PE_5Y', 'score_PB_5Y', 'score_PE_sector',
    'score_PB_sector', 'score_valuation', 'score_final'
]].astype(float)

df_bank['score_profit'] = round((df_bank['score_ROE_sector'] + df_bank['score_ROA_sector'] + df_bank['score_NIM_sector'])*4/3,2)

for _, items in df_bank.iterrows():
    if items['score_profit'] < 1:
        items['rank_profit'] = 'D'
    elif items['score_profit'] < 2:
        items['rank_profit'] = 'C'
    elif items['score_profit'] < 3:
        items['rank_profit'] = 'B'
    else:
        items['rank_profit'] = 'A'
        
df_bank['score_final'] = round(np.mean(df_bank[['score_profit', 'score_health', 'score_growth', 'score_valuation']], axis=1),2)

for _, items in df_bank.iterrows():
    if items['score_final'] < 1:
        items['rank_final'] = 'D'
    elif items['score_final'] < 2:
        items['rank_final'] = 'C'
    elif items['score_final'] < 3:
        items['rank_final'] = 'B'
    else:
        items['rank_final'] = 'A'
        

df_bank = df_bank.astype(str)

In [37]:
df_bank

,Symbol,Year,Quarter,score_ROE_sector,score_ROA_sector,score_NIM_sector,score_profit,rank_profit,z_LoanProvisionRatio,z_Deposit2Loan,...,rank_growth,score_PE_5Y,score_PB_5Y,score_PE_sector,score_PB_sector,score_valuation,rank_valuation,score_final,rank_final,Update
5,ABB,2021,4,0.0,1.0,0.0,1.33,B,0.0,2.0,...,C,0.0,0.0,0.0,0.0,0.0,D,0.83,C,20220422
11,ACB,2021,4,1.0,1.0,1.0,4.0,A,3.0,2.0,...,D,1.0,1.0,0.0,1.0,1.5,C,2.19,B,20220422
49,BAB,2021,4,0.0,0.0,0.0,0.0,C,2.0,3.0,...,C,0.0,0.0,0.0,0.0,0.0,D,1.0,C,20220422
69,BID,2021,4,0.0,0.0,1.0,1.33,C,4.0,2.0,...,A,1.0,1.0,0.0,0.0,1.0,D,2.08,B,20220422
103,BVB,2021,4,0.0,0.0,0.0,0.0,C,2.0,1.0,...,D,0.0,0.0,0.0,0.0,0.0,D,0.38,D,20220422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17957,VAB,2023,1,0.0,0.0,0.0,0.0,C,0.0,4.0,...,D,0.0,0.0,0.0,0.0,0.0,D,0.56,C,20230627
17960,VBB,2023,1,0.0,0.0,0.0,0.0,C,0.0,4.0,...,A,0.0,0.0,2.0,0.0,1.0,D,1.25,C,20230627
17985,VIB,2023,1,1.0,1.0,1.0,4.0,A,2.0,1.0,...,C,2.0,2.0,0.0,0.0,2.0,C,1.94,C,20230627
18023,VCB,2023,1,1.0,1.0,0.0,2.67,B,4.0,3.0,...,A,2.0,2.0,0.0,0.0,2.0,C,2.86,B,20230627


In [38]:
select_column = df_bank.columns[:-1].to_list()

In [39]:
df_bank[select_column]

,Symbol,Year,Quarter,score_ROE_sector,score_ROA_sector,score_NIM_sector,score_profit,rank_profit,z_LoanProvisionRatio,z_Deposit2Loan,...,score_growth,rank_growth,score_PE_5Y,score_PB_5Y,score_PE_sector,score_PB_sector,score_valuation,rank_valuation,score_final,rank_final
5,ABB,2021,4,0.0,1.0,0.0,1.33,B,0.0,2.0,...,1.0,C,0.0,0.0,0.0,0.0,0.0,D,0.83,C
11,ACB,2021,4,1.0,1.0,1.0,4.0,A,3.0,2.0,...,0.0,D,1.0,1.0,0.0,1.0,1.5,C,2.19,B
49,BAB,2021,4,0.0,0.0,0.0,0.0,C,2.0,3.0,...,1.0,C,0.0,0.0,0.0,0.0,0.0,D,1.0,C
69,BID,2021,4,0.0,0.0,1.0,1.33,C,4.0,2.0,...,3.0,A,1.0,1.0,0.0,0.0,1.0,D,2.08,B
103,BVB,2021,4,0.0,0.0,0.0,0.0,C,2.0,1.0,...,0.0,D,0.0,0.0,0.0,0.0,0.0,D,0.38,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17957,VAB,2023,1,0.0,0.0,0.0,0.0,C,0.0,4.0,...,0.0,D,0.0,0.0,0.0,0.0,0.0,D,0.56,C
17960,VBB,2023,1,0.0,0.0,0.0,0.0,C,0.0,4.0,...,3.0,A,0.0,0.0,2.0,0.0,1.0,D,1.25,C
17985,VIB,2023,1,1.0,1.0,1.0,4.0,A,2.0,1.0,...,1.0,C,2.0,2.0,0.0,0.0,2.0,C,1.94,C
18023,VCB,2023,1,1.0,1.0,0.0,2.67,B,4.0,3.0,...,3.0,A,2.0,2.0,0.0,0.0,2.0,C,2.86,B


## 2. Save data to DB Access

### 2.1 Get data fields in new table
- `ptsp_stock_fundamental_score_financial`

In [40]:
conn = pyodbc.connect(
    r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=V:\iBroker\stock_database.accdb;'
)
df_db = pd.read_sql("SELECT * FROM ptsp_stock_fundamental_score_financial", con=conn)
conn.commit()

In [41]:
df_db.rename(
    columns={
        "score_roe_sector": "score_ROE_sector",
        "score_roa_sector": "score_ROA_sector",
        "score_nim_sector": "score_NIM_sector"
    },
    inplace=True
)

In [42]:
df_saving = pd.concat(
    [
        df_db[select_column],
        df_bank[select_column].astype(str)
    ]
).drop_duplicates(keep=False)

In [43]:
df_saving['Update'] = now

In [44]:
# "],[".join(i for i in df_db.columns.to_list())
col_df_db = '[Symbol],[Year],[Quarter],[score_roe_sector],[score_roa_sector],[score_nim_sector],[score_profit],[rank_profit],[z_LoanProvisionRatio],[z_Deposit2Loan],[z_NPL_ratio_inv],[z_NPL_coverage],[score_health],[rank_health],[score_EPS_above_average],[score_EPS_growth],[score_EPS_above_sector],[score_EPS_above_group],[score_growth],[rank_growth],[score_PE_5Y],[score_PB_5Y],[score_PE_sector],[score_PB_sector],[score_valuation],[rank_valuation],[score_final],[rank_final],[update]'

### 2.2 Save data to new table

In [45]:
conn = pyodbc.connect(
    r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=V:\iBroker\stock_database.accdb;'
)
cursor = conn.cursor()
for _, row in df_bank.iterrows():
    sql = "INSERT INTO ptsp_stock_fundamental_score_financial ("+col_df_db+") VALUES "+ str(tuple(row))
    cursor.execute(sql)
    conn.commit()

conn.close()
print("Successfully saved data")


Successfully saved data
